# Подключение к web-службе Azure ML

__Цель лабораторной работы:__

- научиться подключаться в web-службам, используя Конечные точки (Endpoints) Azure ML
- научиться получать прогнозы для новых данных.

## Подготовка среды

Импорт необходимых модулей и проверка версии Azure ML SDK:

In [ ]:
import urllib.request
import json
import os
import ssl

Получим конфигурацию эксперимента:

In [ ]:
%run core.py

# WARN: replace the value of following parameter with your own in the config.yml:
# - endpoint_uri
# - api_key

config = get_experiment_config('lab_3B')
config

## Входные данные

Формирование входных данных в JSON формате:


In [ ]:
data = {
    "Inputs": {
          "WebServiceInput0":
          [
              {
                    'Pregnancies': "6",
                    'Glucose': "148",
                    'BloodPressure': "72",
                    'SkinThickness': "35",
                    'Insulin': "0",
                    'BMI': "33.6",
                    'DiabetesPedigreeFunction': "0.627",
                    'Age': "50",
                    'Outcome': "1",
              },
          ],
    },
    "GlobalParameters":  {
    }
}

data

## Подключение к web-службе

Установка URL и секретного ключа (API Key) web-службы:

In [ ]:
url = config['endpoint_uri']
api_key = config['api_key']

Запрос к службе и получение ответа:

In [ ]:
def allowSelfSignedHttps(allowed):
    # bypass the server certificate verification on client side
    if allowed and not os.environ.get('PYTHONHTTPSVERIFY', '') and getattr(ssl, '_create_unverified_context', None):
        ssl._create_default_https_context = ssl._create_unverified_context

allowSelfSignedHttps(True) # this line is needed if you use self-signed certificate in your scoring service.


body = str.encode(json.dumps(data))

headers = {'Content-Type':'application/json', 'Authorization':('Bearer '+ api_key)}

req = urllib.request.Request(url, body, headers)

try:
    response = urllib.request.urlopen(req)

    result = response.read()
    print(result)
except urllib.error.HTTPError as error:
    print('Request failed. Status code: ' + str(error.code))

    # Print the headers - they include the requert ID and the timestamp, which are useful for debugging the failure
    print(error.info())
    print(json.loads(error.read().decode('utf8', 'ignore')))


Поэкспериментируйте с запросами в web-службе, подставляя в data свои собственные значения полей.